In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import IPython.html.widgets as widgets

In [ ]:
import scipy
from scipy.stats import norm
import scipy.optimize as op
import emcee
import triangle

In [ ]:
seaborn.set_context('talk')

In [ ]:
import simtransient as st
import simtransient.utils as stutils
from simtransient import hammer
from simtransient.models.supernova import Sn1aOpticalEnsemble, Sn2OpticalEnsemble
from simtransient.modelrun import ModelRun

In [ ]:
mr1 = ModelRun(ensemble=Sn1aOpticalEnsemble(),
              emcee_kwargs=dict(threads=4))
mr2 = ModelRun(ensemble=Sn2OpticalEnsemble(),
              emcee_kwargs=dict(threads=4))

In [ ]:
mr1.sample(300)

In [ ]:
mr2.sample(300)

In [ ]:
mr=mr2

In [ ]:
mr.plot_walkers()

In [ ]:
mr.plot_hists()

In [ ]:
mr.plot_triangle()

In [ ]:
seaborn.palplot(seaborn.color_palette('Paired'))

In [ ]:
palette = seaborn.color_palette('Paired')
pal1 = dict(trace=palette[0], fo

In [ ]:
tsteps=np.linspace(-30,80,500)
maxprior_pars = mr.ensemble.gauss_pars.T.mu.copy()
maxprior_pars['t0']=0
maxprior = mr.ensemble.get_curve(**maxprior_pars)
mr2.plot_forecast(tsteps, 
                  t_forecast=20,
#                  true_curve = maxprior
                 )

In [ ]:
def plt_traces(ntrace):
#     ntrace=int(ntrace)
    for modelname, model in models.iteritems():
        if model.plotthis:
            choice_idx=np.random.choice(len(model.trimmed), size=ntrace, replace=False)
            subsamples = model.trimmed[choice_idx]

            t=np.linspace(-30, 80,1000)
            lcs=[]
            for pltpars in subsamples:
                lcs.append(model.ensemble.evaluate(t,*pltpars,t0=0))
            lcs=np.array(lcs)
        #     seaborn.tsplot( lcs,  t, err_style="unit_traces", ls='', color='y')
            seaborn.tsplot( lcs,  t, err_style="unit_traces", ls='', 
                           value='flux',
                           color=model.color,
                           )

In [ ]:
models.loc['plotthis']=True
# models.sn1.plotthis=False
# models.sn2.plotthis=False

# widgets.interact(
widgets.interact_manual(
    plt_traces, ntrace=(0,150),
        )